## Understanding the relationship between CERC and CIP participants

Let's start extracting the list of participants from publicly available lists starting with CERC.

In [97]:
import tabula
import pandas as pd

df_cerc = tabula.read_pdf("./input/LISTA-PARTICIPANTES-CERC-ARRANJOS-DE-PAGAMENTO-1.pdf", pages="all")

# In first dataframe transform the first row in header
df_cerc[0].columns = df_cerc[0].iloc[0]
df_cerc[0] = df_cerc[0].drop(df_cerc[0].index[0])

# Transform the header in the first row
for i in range(1,7):
    df_cerc[i].loc[-1] = df_cerc[i].columns
    df_cerc[i].index = df_cerc[i].index + 1
    df_cerc[i].sort_index(inplace=True)
    df_cerc[i].columns = df_cerc[0].columns

df_cerc = pd.concat([df_cerc[0], df_cerc[1], df_cerc[2], df_cerc[3], df_cerc[4], df_cerc[5], df_cerc[6]], ignore_index=True)

# Remove the first column in all the dataframes
df_cerc = df_cerc.drop(df_cerc.columns[:1], axis=1)

df_cerc.reset_index(drop=True)

# Renaming the columns
df_cerc.columns = ["corporate_name", "cnpj", "profile"]

# Remove dashes and dots in cnpj column
df_cerc["cnpj"] = df_cerc["cnpj"].str.replace("[./-]", "", regex=True)



The same operation we perform with CIP:

In [253]:
df_cip = tabula.read_pdf("./input/Participantes Homologados.pdf", pages="all")

# Remove first three rows from the first page
df_cip[0] = df_cip[0].drop(df_cip[0].index[:3])
df_cip[4] = df_cip[4].drop(df_cip[4].index[:3])


In [254]:
# Remove unnecessary columns from the first page and forth page
df_cip[0] = df_cip[0].drop(df_cip[0].columns[[0,1,12]], axis=1)
df_cip[4] = df_cip[4].drop(df_cip[4].columns[[0]], axis=1)

# Naming the columns correctly
df_cip[0].columns = ["cnpj", "ispb", "corporate_name", "is_iap", "is_acquirer", \
    "is_bank_account_owner", "is_issuer", "is_settler", "is_subacquirer", "is_subacquirer_receiver"]
df_cip[4].columns = df_cip[0].columns

# Handling all the pages at one expcept the 4th page (!!!)
for i in (1,2,3,5):
    df_cip[i] = df_cip[i].drop(df_cip[i].index[:2])
    df_cip[i] = df_cip[i].drop(df_cip[i].columns[[0,1,12]], axis=1)
    df_cip[i].columns = ["cnpj", "ispb", "corporate_name", "is_iap", "is_acquirer", \
        "is_bank_account_owner", "is_issuer", "is_settler", "is_subacquirer", "is_subacquirer_receiver"]

df_cip = pd.concat([df_cip[0], df_cip[1], df_cip[2], df_cip[3], df_cip[4], df_cip[5]], ignore_index=True)

df_cip["cnpj"] = df_cip["cnpj"].str.replace("[./-]", "", regex=True)

In [255]:
# Merging CIP and CERC dataframes
df_merged = pd.merge(df_cerc, df_cip, on="cnpj", how="inner")

In [267]:
df_merged.info()
df_merged.fillna({'is_iap':False, 'is_acquirer':False, 'is_bank_account_owner':False, 'is_issuer':False,\
    'is_settler':False, 'is_subacquirer': False, 'is_subacquirer_receiver':False}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 0 to 162
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   corporate_name_x         163 non-null    object
 1   cnpj                     163 non-null    object
 2   profile                  163 non-null    object
 3   ispb                     163 non-null    object
 4   corporate_name_y         163 non-null    object
 5   is_iap                   10 non-null     object
 6   is_acquirer              57 non-null     object
 7   is_bank_account_owner    33 non-null     object
 8   is_issuer                34 non-null     object
 9   is_settler               16 non-null     object
 10  is_subacquirer           60 non-null     object
 11  is_subacquirer_receiver  70 non-null     object
dtypes: object(12)
memory usage: 16.6+ KB


In [268]:
df_sub = df_merged.loc[(df_merged['is_subacquirer'] == "X") | (df_merged['is_subacquirer_receiver'] == "X")]

In [270]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 162
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   corporate_name_x         117 non-null    object
 1   cnpj                     117 non-null    object
 2   profile                  117 non-null    object
 3   ispb                     117 non-null    object
 4   corporate_name_y         117 non-null    object
 5   is_iap                   117 non-null    object
 6   is_acquirer              117 non-null    object
 7   is_bank_account_owner    117 non-null    object
 8   is_issuer                117 non-null    object
 9   is_settler               117 non-null    object
 10  is_subacquirer           117 non-null    object
 11  is_subacquirer_receiver  117 non-null    object
dtypes: object(12)
memory usage: 11.9+ KB


In [271]:
df_acquirers = df_cip.loc[df_cip['is_acquirer'] == "X"]

In [269]:
df_sub.to_csv("./output/subacquirers_br.csv", index=False)